In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import GridSearchCV
import joblib

In [ ]:
# Load the dataset
data = pd.read_csv('creditcard.csv')

In [ ]:
# Display the first few rows of the dataset
print(data.head())

In [ ]:
# Check for missing values
print(data.isnull().sum())

In [ ]:
# Check class distribution
print(data['Class'].value_counts())

In [ ]:
# Visualize class distribution
plt.figure(figsize=(8, 6))
sns.countplot(x='Class', data=data)
plt.title('Class Distribution')
plt.xlabel('Class')
plt.ylabel('Count')
plt.show()

In [ ]:
# Separate features and target variable
X = data.drop(columns=['Class'])
y = data['Class']

In [ ]:
# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# Balancing the data using SMOTE
smote = SMOTE(random_state=42)
X_train_balanced, y_train_balanced = smote.fit_resample(X_train, y_train)

In [ ]:
# Implement Hyperparameter Tuning/Model Improvement
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}
grid_search = GridSearchCV(estimator=RandomForestClassifier(random_state=42),
                           param_grid=param_grid,
                           cv=5,
                           scoring='accuracy',
                           n_jobs=-1)
grid_search.fit(X_train_balanced, y_train_balanced)

In [ ]:
# Print the best hyperparameters and the corresponding accuracy
print("Best Hyperparameters:", grid_search.best_params_)
print("Best Accuracy:", grid_search.best_score_)

In [ ]:
# Train the final model with the best hyperparameters
best_rf_classifier = RandomForestClassifier(**grid_search.best_params_, random_state=42)
best_rf_classifier.fit(X_train_balanced, y_train_balanced)

In [ ]:
# Evaluate the final model
y_pred = best_rf_classifier.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy Score:", accuracy)
print("Classification Report:")
print(classification_report(y_test, y_pred))

In [ ]:
# Save the trained model
joblib.dump(best_rf_classifier, 'credit_card_fraud_detection_model.pkl')